### Abrindo o vídeo e mostrando o vídeo

In [1]:
import numpy as np
import matplotlib
matplotlib.use('TkAgg')
import matplotlib.pyplot as plt
from IPython import display
from time import sleep

import pims
from PIL import Image

v = pims.Video('../dataset/25_20160407_front/normal/crowd/2016_04_07_06_49_39FrontColor.avi')

In [2]:
def play(video):
    fig = plt.figure()
    ax = fig.gca()
    fig.show()

    for ii in range(len(video)):
        im = video[ii]
        plt.imshow(im, cmap='gray')
        ax.set_title(ii)

        fig.canvas.draw()
        sleep(0.1)

In [23]:
def play_cmp(video1, video2):
    fig = plt.figure()
    fig1 = fig.add_subplot(121)
    fig2 = fig.add_subplot(122)
    ax = fig.gca()
    fig.show()

    for ii in range(len(video1)):
        im1 = video1[ii]
        im2 = video2[ii]
        fig1.imshow(im1, cmap='gray')
        fig2.imshow(im2, cmap='gray')
        ax.set_title(ii)

        fig.canvas.draw()
        sleep(0.1)

Algumas informações sobre o vídeo aberto:

In [3]:
print(v.get_metadata())

v_array = np.array(v)
print('\n', v_array.shape)

{'plugin': 'ffmpeg', 'nframes': 322, 'ffmpeg_version': 'N-86111-ga441aa90e8-static http://johnvansickle.com/ffmpeg/ built with gcc 5.4.1 (Debian 5.4.1-8) 20170304', 'fps': 25.0, 'source_size': (320, 240), 'size': (320, 240), 'duration': 12.88}

 (322, 240, 320, 3)


Os campos da *tupla* anterior significam o seguinte:
(Frames, Altura, Largura, Cores)

### Convertendo o vídeo para escala de cinza

In [4]:
def to_gray(video):
    frames, width, hight, color = video.shape
    video_gray = np.zeros((frames, width, hight), dtype='uint32')
    
    for ii in range(len(video)):
        r, g, b = Image.fromarray(video[ii]).split()
        
        r = np.array(r, dtype='uint32')
        g = np.array(g, dtype='uint32')
        b = np.array(b, dtype='uint32')
        
        video_gray[ii] = (r + g + b) / 3
        
    return video_gray

In [5]:
I = to_gray(v_array)
I.shape
# play(x[0:30])

(322, 240, 320)

### Background Modeling Independent of Time

In [18]:
tao = 2147483647

In [25]:
B_BMIT = I[0]
D_BMIT = np.abs(I - B_BMIT)

# F_BMIT = D_BMIT
# F_BMIT[F_BMIT >= tao] = True
# F_BMIT[F_BMIT < tao] = False

# play_cmp(I, D_BMIT)

###  The Long Term Average Background Modeling

In [7]:
B_LTAMBM = np.zeros(I.shape)
for k in range(1, len(I)):
    B_LTAMBM[k] = (1-1/k)*B_LTAMBM[k-1] + (1/k) *I[k]

D_LTAMBM = np.abs(I - B_LTAMBM)

### Moving Average Backgound Modeling

In [12]:
W = 5
B_MABM = np.zeros(I.shape)
for k in range((W-1), len(I)-1):
    B_MABM[k] = (1/W)*np.sum(I[(k-(W-1)):(k+1),:,:], axis=0)

D_MABM = np.abs(I - B_MABM)
